<a href="https://colab.research.google.com/github/myoshi1212/quantmacro/blob/main/GrothAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np

def calculate_average_growth_rate(data, start_year, end_year, column):
    start_value = data.loc[data['year'] == start_year, column].iloc[0]
    end_value = data.loc[data['year'] == end_year, column].iloc[0]
    years = end_year - start_year
    avg_growth_rate = np.exp(np.log(end_value / start_value) / years) - 1
    return avg_growth_rate * 100  # Convert to percentage

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'countrycode', 'country', 'rgdpna', 'rnna', 'emp', 'avh', 'labsh', 'hc']]
data = data.loc[(data['year'] >= 1975) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['k_n'] = data['rnna'] / data['emp']   # K/L
data['hc_n'] = data['hc']
data['hours_n'] = data['avh']

# Initialize results list
results = []

# Calculate growth rate for each country and store results
for country in countries:
    country_data = data[data['country'] == country]
    if country_data.empty:
        print(f"No data for {country}")
        continue

    alpha = country_data['alpha'].mean()
    gdp_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'y_n')
    capital_deepening = calculate_average_growth_rate(country_data, 1975, 2019, 'k_n') * alpha
    hc_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'hc_n')
    hours_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'hours_n')
    labor_growth = (1 - alpha) * (hc_growth + hours_growth)
    tfp_growth = gdp_growth - capital_deepening - labor_growth
    tfp_share = tfp_growth / gdp_growth * 100  # Convert to percentage
    capital_share = capital_deepening / gdp_growth * 100  # Convert to percentage

    results.append({
        'Country': country,
        'Growth Rate': gdp_growth,
        'TFP Growth': tfp_growth,
        'Capital Deepening': capital_deepening,
        'TFP Share': tfp_share,
        'Capital Share': capital_share
    })

# Create DataFrame from results
growth_accounting_table = pd.DataFrame(results)

# Calculate averages for each column
average_row = growth_accounting_table.mean(numeric_only=True).to_dict()
average_row['Country'] = 'Average'

# Convert the average row dictionary to a DataFrame
average_df = pd.DataFrame([average_row])

# Concatenate the original DataFrame with the average row DataFrame
growth_accounting_table = pd.concat([growth_accounting_table, average_df], ignore_index=True)

print(growth_accounting_table)


           Country  Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
0        Australia     1.191826    0.692145           0.433507  58.074347   
1          Austria     1.455530    0.590339           0.785567  40.558343   
2          Belgium     1.388993    0.499884           0.688190  35.988939   
3           Canada     0.929646    0.188247           0.545347  20.249331   
4          Denmark     1.503476    0.785171           0.606466  52.223698   
5          Finland     1.890487    0.886188           0.822171  46.876157   
6           France     1.362772    0.705003           0.565098  51.733040   
7          Germany     1.470326    0.995389           0.590177  67.698538   
8           Greece     0.724438   -0.423439           0.765961 -58.450699   
9          Iceland     1.839406    1.245772           0.404489  67.726851   
10         Ireland     3.218370    1.630199           1.575644  50.652951   
11           Italy     0.994157   -0.178337           0.834675 -17.938515   